This code goes from Topcoder's winners page, to the problem statement, to the problem detail of each SRM

In [ ]:
import requests, json, sys, time, re
from bs4 import BeautifulSoup
key = 'round_overview'
number = 1
counter = 0
myLinks = []
while(1):
    URL = "http://community.topcoder.com/tc?module=SrmDivisionWins&sc=&sd=&nr=200&sr=" + str(number)
    number += 200
    r = requests .get(URL)
    t = r.text
    
    if ("An error has occurred when attempting to process your request" in t):
        print("There were no results matching your query.")
        print(counter)
        sys.exit()
    soup = BeautifulSoup(t, 'html.parser')
    start = 0
    npages = 1
    for link in soup.findAll('a'):
        s = link.get('href')
        
        if("topcoder" in s):
            continue;
        
        # Print the Round overview urls
        if(key in str(s)):
            counter += 1;
            #print ("http://community.topcoder.com" + s)
            myLinks.append("http://community.topcoder.com" + s)



In [ ]:
import requests, json, sys, time, re
from bs4 import BeautifulSoup
key = 'problem_statement'
number = 1
counter = 0
problemStatements = []
for Link in myLinks:
    URL = Link
    r = requests .get(URL)
    t = r.text

    soup = BeautifulSoup(t, 'html.parser')
    start = 0
    npages = 1
    for link in soup.findAll('a'):
        s = link.get('href')
    
        # Print the Problem Statement urls
        if(key in str(s)):
            counter += 1;
            #print ("http://community.topcoder.com" + s)
            problemStatements.append("http://community.topcoder.com" + s)


This portion never would finish running on Jupyter, so I ran it on hydra instead. The only change I made was putting the links in a set to strip any duplicates

In [ ]:
import requests, json, sys, time, re
from bs4 import BeautifulSoup
key = 'ProblemDetail'
number = 1
counter = 0
problemDetails = []
for Statement in problemStatements:
    URL = Statement
    r = requests .get(URL)
    t = r.text

    soup = BeautifulSoup(t, 'html.parser')
    start = 0
    npages = 1
    for link in soup.findAll('a'):
        s = link.get('href')
    
        # Print the Problem Detail urls
        if(key in str(s)):
            counter += 1;
            #print ("http://community.topcoder.com" + s)
            problemDetails.append("http://community.topcoder.com" + s)

This portion parses each url and grabs the data. It then stores it into MongoDB for accessing later.

In [ ]:
import requests, json, re, pprint, pymongo
from bs4 import BeautifulSoup
key = 'ProblemDetail'
number = 1
counter = 0
#problemDetails = []

def cleanHtml (html):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    return soup .get_text()

f = open('TopcoderDetailsList', 'r')
client = pymongo.MongoClient (host="da0.eecs.utk.edu")
db = client['TopCoder']
coll = db['challenges']
count = 0
for line in f:
    count += 1
    print(count)
    URL = line;
    r = requests .get(URL)
    t = r.content
    t = cleanHtml(t)
    i = re.IGNORECASE
    problemInfo = dict()
    
    # Not sure why, but a few links aren't valid 
    if("An error has occurred when attempting to process your request" in t):
        continue
    
    # grab problem name
    try:
        problemInfo['problem_name'] = re.search('(problem name:\s+(.*))', t, i).group(2).replace("\n", "")
    except:
        continue
    
    # grab used in
    problemInfo['used_in'] = re.search('used in:\s+(.*)', t, i).group(1)
    
    # grab categories
    problemInfo['categories'] = re.search('categories:\s+(.*)', t, i).group(1)
    
    # grab writer
    problemInfo['writer'] = re.search('writer:\s+(.*)', t, i).group(1)
    
    # grab testers
    try:
        testers = re.search('testers?:\s+([A-Za-z0-9_ \.,\n\[\]]+?)division', t, i).group(1)
        testers = re.sub('\s+', ' ', testers).strip().replace("\n", "").replace(' ,', ',')
    except:
        continue
        
    problemInfo['testers'] = testers;

    
    m = re.search('used as:\s+(division i{1,2} level (one|two|three)\s+,\s+division i{1,2} level (one|two|three))', t, i)
    if m: 
        # handle two divisions
        divisionStr = m.group(1)
        t = re.sub('used as:\s+(division i{1,2} level (one|two|three)\s+,\s+division i{1,2} level (one|two|three))', '', t, flags=i)
        
        divisionStr = re.sub('\s+', ' ', divisionStr).strip().replace("\n", "")
        problemInfo['used_as'] = divisionStr
        
        # grab point value
        try:
            pointValue = re.search('point value\s+(\d+)\s+(\d+)', t, i)
            
            problemInfo['div_i_point_value'] = int(pointValue.group(1))
            problemInfo['div_ii_point_value'] = int(pointValue.group(2))
        except:
            # the challenge was not completed. Move on.
            continue
        
        # grab competitors
        competitors = re.search('competitors\s+(\d+)\s+(\d+)', t, i)
        problemInfo['div_i_competitors'] = int(competitors.group(1))
        problemInfo['div_ii_competitors'] = int(competitors.group(2))
        
        # grab opens
        opens = re.search('opens\s+(\d+)\s+(\d+)', t, i)
        problemInfo['div_i_opens'] = int(opens.group(1))
        problemInfo['div_ii_opens'] = int(opens.group(2))
        
        # grab percent open
        percentOpen = re.search('percent open\s+(\d+\.\d+)%\s+(\d+\.\d+)%', t, i)
        problemInfo['div_i_percent_open'] = float(percentOpen.group(1))
        problemInfo['div_ii_percent_open'] = float(percentOpen.group(2))
        
        # grab percent submitted
        percentSubmitted = re.search('percent submitted\s+(\d+\.\d+)%\s+(\d+\.\d+)%', t, i)
        problemInfo['div_i_percent_submitted'] = float(percentSubmitted.group(1))
        problemInfo['div_ii_percent_submitted'] = float(percentSubmitted.group(2))
        
        # grab overall accuracy        
        overallAccuracy = re.search('overall accuracy\s+(\d+\.\d+)%\s+(\d+\.\d+)%', t, i)
        problemInfo['div_i_overall_accuracy'] = float(overallAccuracy.group(1)) 
        problemInfo['overall_accuracy'] = float(overallAccuracy.group(2))
        
        # grab division 1 problems submitted
        ps = re.search('division i(?:.|\s)*?problems submitted\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['div_i_problems_submitted_java'] = int(ps.group(1))
        problemInfo['div_i_problems_submitted_c++'] = int(ps.group(2))
        problemInfo['div_i_problems_submitted_c#'] = int(ps.group(3))
        problemInfo['div_i_problems_submitted_vb'] = int(ps.group(4))
        problemInfo['div_i_problems_submitted_python'] = int(ps.group(5))
        problemInfo['div_i_problems_submitted_overall'] = int(ps.group(6))
        
        # grab divison 1 problems correct
        pc = re.search('division i(?:.|\s)*?problems correct\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['div_i_problems_correct_java'] = int(pc.group(1))
        problemInfo['div_i_problems_correct_c++'] = int(pc.group(2))
        problemInfo['div_i_problems_correct_c#'] = int(pc.group(3))
        problemInfo['div_i_problems_correct_vb'] = int(pc.group(4))
        problemInfo['div_i_problems_correct_python'] = int(pc.group(5))
        problemInfo['div_i_problems_correct_overall'] = int(pc.group(6))
        
        # grab division 1 submission accuracy
        sa = re.search('division i(?:.|\s)*?submission accuracy\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?', t, i)
        problemInfo['div_i_submission_accuracy_java'] = float(sa.group(1))
        problemInfo['div_i_submission_accuracy_c++'] = float(sa.group(2))
        problemInfo['div_i_submission_accuracy_c#'] = float(sa.group(3))
        problemInfo['div_i_submission_accuracy_vb'] = float(sa.group(4))
        problemInfo['div_i_submission_accuracy_python'] = float(sa.group(5))
        problemInfo['div_i_submission_accuracy_overall'] = float(sa.group(6))
        
        # grab division 1 problems failed by challenge
        pfbc = re.search('division i(?:.|\s)*?problems failed by challenge\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['div_i_problems_failed_by_challenge_java'] = int(pfbc.group(1))
        problemInfo['div_i_problems_failed_by_challenge_c++'] = int(pfbc.group(2))
        problemInfo['div_i_problems_failed_by_challenge_c#'] = int(pfbc.group(3))
        problemInfo['div_i_problems_failed_by_challenge_vb'] = int(pfbc.group(4))
        problemInfo['div_i_problems_failed_by_challenge_python'] = int(pfbc.group(5))
        problemInfo['div_i_problems_failed_by_challenge_overall'] = int(pfbc.group(6))
        
        # grab division 1 problems failed by system test
        pfbst = re.search('division i(?:.|\s)*?problems failed by system test\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['div_i_problems_failed_by_system_test_java'] = int(pfbst.group(1))
        problemInfo['div_i_problems_failed_by_system_test_c++'] = int(pfbst.group(2))
        problemInfo['div_i_problems_failed_by_system_test_c#'] = int(pfbst.group(3))
        problemInfo['div_i_problems_failed_by_system_test_vb'] = int(pfbst.group(4))
        problemInfo['div_i_problems_failed_by_system_test_python'] = int(pfbst.group(5))
        problemInfo['div_i_problems_failed_by_system_test_overall'] = int(pfbst.group(6))
        
        # grab division 1 challenge attempts made
        cam = re.search('division i(?:.|\s)*?challenge attempts made\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['div_i_challenge_attempts_made_java'] = int(cam.group(1))
        problemInfo['div_i_challenge_attempts_made_c++'] = int(cam.group(2))
        problemInfo['div_i_challenge_attempts_made_c#'] = int(cam.group(3))
        problemInfo['div_i_challenge_attempts_made_vb'] = int(cam.group(4))
        problemInfo['div_i_challenge_attempts_made_python'] = int(cam.group(5))
        problemInfo['div_i_challenge_attempts_made_overall'] = int(cam.group(6))
        
        # grab division 1 challenge accuracy
        ca = re.search('division i(?:.|\s)*?challenge accuracy\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?', t, i)
        problemInfo['div_i_challenge_accuracy_java'] = float(ca.group(1))
        problemInfo['div_i_challenge_accuracy_c++'] = float(ca.group(2))
        problemInfo['div_i_challenge_accuracy_c#'] = float(ca.group(3))
        problemInfo['div_i_challenge_accuracy_vb'] = float(ca.group(4))
        problemInfo['div_i_challenge_accuracy_python'] = float(ca.group(5))
        problemInfo['div_i_challenge_accuracy_overall'] = float(ca.group(6))
        
        # grab division 1 best time
        try:
            bt = re.search('division i(?:.|\s)*?best time\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})', t, i)
            problemInfo['div_i_best_time_java'] = bt.group(1)
            problemInfo['div_i_best_time_c++'] = bt.group(2)
            problemInfo['div_i_best_time_c#'] = bt.group(3)
            problemInfo['div_i_best_time_vb'] = bt.group(4)
            problemInfo['div_i_best_time_python'] = bt.group(5)
            problemInfo['div_i_best_time_overall'] = bt.group(6)
        except: # no best time for any category
            problemInfo['div_i_best_time_java'] = None
            problemInfo['div_i_best_time_c++'] = None
            problemInfo['div_i_best_time_c#'] = None
            problemInfo['div_i_best_time_vb'] = None
            problemInfo['div_i_best_time_python'] = None
            problemInfo['div_i_best_time_overall'] = None
        
        # grab division 1 average correct time
        try:
            act = re.search('division i(?:.|\s)*?average correct time\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})', t, i)
            problemInfo['div_i_average_correct_time_java'] = act.group(1)
            problemInfo['div_i_average_correct_time_c++'] = act.group(2)
            problemInfo['div_i_average_correct_time_c#'] = act.group(3)
            problemInfo['div_i_average_correct_time_vb'] = act.group(4)
            problemInfo['div_i_average_correct_time_python'] = act.group(5)
            problemInfo['div_i_average_correct_time_overall'] = act.group(6)
        except: # whole line is empty
            problemInfo['div_i_average_correct_time_java'] = None
            problemInfo['div_i_average_correct_time_c++'] = None
            problemInfo['div_i_average_correct_time_c#'] = None
            problemInfo['div_i_average_correct_time_vb'] = None
            problemInfo['div_i_average_correct_time_python'] = None
            problemInfo['div_i_average_correct_time_overall'] = None
        
        # remove last conflicting text
        t = re.sub('division i\s+division ii\s+point value', '', t, flags=i)
    
        ### DIVISION 2 ###
         # grab division 2 problems submitted
        ps = re.search('division ii(?:.|\s)*?problems submitted\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['div_ii_problems_submitted_java'] = int(ps.group(1))
        problemInfo['div_ii_problems_submitted_c++'] = int(ps.group(2))
        problemInfo['div_ii_problems_submitted_c#'] = int(ps.group(3))
        problemInfo['div_ii_problems_submitted_vb'] = int(ps.group(4))
        problemInfo['div_ii_problems_submitted_python'] = int(ps.group(5))
        problemInfo['div_ii_problems_submitted_overall'] = int(ps.group(6))
        
        # grab divison 2 problems correct
        pc = re.search('division ii(?:.|\s)*?problems correct\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['div_ii_problems_correct_java'] = int(pc.group(1))
        problemInfo['div_ii_problems_correct_c++'] = int(pc.group(2))
        problemInfo['div_ii_problems_correct_c#'] = int(pc.group(3))
        problemInfo['div_ii_problems_correct_vb'] = int(pc.group(4))
        problemInfo['div_ii_problems_correct_python'] = int(pc.group(5))
        problemInfo['div_ii_problems_correct_overall'] = int(pc.group(6))
        
        # grab division 2 submission accuracy
        sa = re.search('division ii(?:.|\s)*?submission accuracy\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?', t, i)
        problemInfo['div_ii_submission_accuracy_java'] = float(sa.group(1))
        problemInfo['div_ii_submission_accuracy_c++'] = float(sa.group(2))
        problemInfo['div_ii_submission_accuracy_c#'] = float(sa.group(3))
        problemInfo['div_ii_submission_accuracy_vb'] = float(sa.group(4))
        problemInfo['div_ii_submission_accuracy_python'] = float(sa.group(5))
        problemInfo['div_ii_submission_accuracy_overall'] = float(sa.group(6))
        
        # grab division 2 problems failed by challenge
        pfbc = re.search('division ii(?:.|\s)*?problems failed by challenge\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['div_ii_problems_failed_by_challenge_java'] = int(pfbc.group(1))
        problemInfo['div_ii_problems_failed_by_challenge_c++'] = int(pfbc.group(2))
        problemInfo['div_ii_problems_failed_by_challenge_c#'] = int(pfbc.group(3))
        problemInfo['div_ii_problems_failed_by_challenge_vb'] = int(pfbc.group(4))
        problemInfo['div_ii_problems_failed_by_challenge_python'] = int(pfbc.group(5))
        problemInfo['div_ii_problems_failed_by_challenge_overall'] = int(pfbc.group(6))
        
        # grab division 2 problems failed by system test
        pfbst = re.search('division ii(?:.|\s)*?problems failed by system test\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['div_ii_problems_failed_by_system_test_java'] = int(pfbst.group(1))
        problemInfo['div_ii_problems_failed_by_system_test_c++'] = int(pfbst.group(2))
        problemInfo['div_ii_problems_failed_by_system_test_c#'] = int(pfbst.group(3))
        problemInfo['div_ii_problems_failed_by_system_test_vb'] = int(pfbst.group(4))
        problemInfo['div_ii_problems_failed_by_system_test_python'] = int(pfbst.group(5))
        problemInfo['div_ii_problems_failed_by_system_test_overall'] = int(pfbst.group(6))
        
        # grab division 2 challenge attempts made
        cam = re.search('division ii(?:.|\s)*?challenge attempts made\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['div_ii_challenge_attempts_made_java'] = int(cam.group(1))
        problemInfo['div_ii_challenge_attempts_made_c++'] = int(cam.group(2))
        problemInfo['div_ii_challenge_attempts_made_c#'] = int(cam.group(3))
        problemInfo['div_ii_challenge_attempts_made_vb'] = int(cam.group(4))
        problemInfo['div_ii_challenge_attempts_made_python'] = int(cam.group(5))
        problemInfo['div_ii_challenge_attempts_made_overall'] = int(cam.group(6))
        
        # grab division 2 challenge accuracy
        ca = re.search('division ii(?:.|\s)*?challenge accuracy\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?', t, i)
        problemInfo['div_ii_challenge_accuracy_java'] = float(ca.group(1))
        problemInfo['div_ii_challenge_accuracy_c++'] = float(ca.group(2))
        problemInfo['div_ii_challenge_accuracy_c#'] = float(ca.group(3))
        problemInfo['div_ii_challenge_accuracy_vb'] = float(ca.group(4))
        problemInfo['div_ii_challenge_accuracy_python'] = float(ca.group(5))
        problemInfo['div_ii_challenge_accuracy_overall'] = float(ca.group(6))
        
        # grab division 2 best time
        try:
            bt = re.search('division ii(?:.|\s)*?best time\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})', t, i)
            problemInfo['div_ii_best_time_java'] = bt.group(1)
            problemInfo['div_ii_best_time_c++'] = bt.group(2)
            problemInfo['div_ii_best_time_c#'] = bt.group(3)
            problemInfo['div_ii_best_time_vb'] = bt.group(4)
            problemInfo['div_ii_best_time_python'] = bt.group(5)
            problemInfo['div_ii_best_time_overall'] = bt.group(6)
        except: # no best time for any category
            problemInfo['div_ii_best_time_java'] = None
            problemInfo['div_ii_best_time_c++'] = None
            problemInfo['div_ii_best_time_c#'] = None
            problemInfo['div_ii_best_time_vb'] = None
            problemInfo['div_ii_best_time_python'] = None
            problemInfo['div_ii_best_time_overall'] = None
        
        # grab division 2 average correct time
        try:
            act = re.search('division ii(?:.|\s)*?average correct time\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})', t, i)
            problemInfo['div_ii_average_correct_time_java'] = act.group(1)
            problemInfo['div_ii_average_correct_time_c++'] = act.group(2)
            problemInfo['div_ii_average_correct_time_c#'] = act.group(3)
            problemInfo['div_ii_average_correct_time_vb'] = act.group(4)
            problemInfo['div_ii_average_correct_time_python'] = act.group(5)
            problemInfo['div_ii_average_correct_time_overall'] = act.group(6)
        except: # whole line is empty
            problemInfo['div_ii_average_correct_time_java'] = None
            problemInfo['div_ii_average_correct_time_c++'] = None
            problemInfo['div_ii_average_correct_time_c#'] = None
            problemInfo['div_ii_average_correct_time_vb'] = None
            problemInfo['div_ii_average_correct_time_python'] = None
            problemInfo['div_ii_average_correct_time_overall'] = None
    else: 
        # handle only one division
        divisionStr = re.search('used as:\s+(division i{1,2} level (one|two|three))', t, i).group(1)
        divisionStr = re.sub('\s+', ' ', divisionStr).strip().replace("\n", "")
        problemInfo['used_as'] = divisionStr
        
        # grab point value
        try:
            pointValue = re.search('point value\s+(\d+)', t, i).group(1)
            problemInfo['point_value'] = int(pointValue.strip())
        except:
            # the challenge was not completed. Move on.
            continue
        
        # grab competitors
        competitors = re.search('competitors\s+(\d+)', t, i).group(1)
        problemInfo['competitors'] = int(competitors)
        
        # grab opens
        opens = re.search('opens\s+(\d+)', t, i).group(1)
        problemInfo['opens'] = int(opens)
        
        # grab percent open
        percentOpen = re.search('percent open\s+(\d+\.\d+)%', t, i).group(1)
        problemInfo['percent_open'] = float(percentOpen)
        
        # grab percent submitted
        percentSubmitted = re.search('percent submitted\s+(\d+\.\d+)%', t, i).group(1)
        problemInfo['percent_submitted'] = float(percentSubmitted)
        
        # grab overall accuracy        
        overallAccuracy = re.search('overall accuracy\s+(\d+\.\d+)%', t, i).group(1)
        problemInfo['overall_accuracy'] = float(overallAccuracy)
        
        # grab problems submitted
        ps = re.search('problems submitted\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['problems_submitted_java'] = int(ps.group(1))
        problemInfo['problems_submitted_c++'] = int(ps.group(2))
        problemInfo['problems_submitted_c#'] = int(ps.group(3))
        problemInfo['problems_submitted_vb'] = int(ps.group(4))
        problemInfo['problems_submitted_python'] = int(ps.group(5))
        problemInfo['problems_submitted_overall'] = int(ps.group(6))
        
        # grab problems correct
        pc = re.search('problems correct\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['problems_correct_java'] = int(pc.group(1))
        problemInfo['problems_correct_c++'] = int(pc.group(2))
        problemInfo['problems_correct_c#'] = int(pc.group(3))
        problemInfo['problems_correct_vb'] = int(pc.group(4))
        problemInfo['problems_correct_python'] = int(pc.group(5))
        problemInfo['problems_correct_overall'] = int(pc.group(6))
        
        # grab submission accuracy
        sa = re.search('submission accuracy\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?', t, i)
        problemInfo['submission_accuracy_java'] = float(sa.group(1))
        problemInfo['submission_accuracy_c++'] = float(sa.group(2))
        problemInfo['submission_accuracy_c#'] = float(sa.group(3))
        problemInfo['submission_accuracy_vb'] = float(sa.group(4))
        problemInfo['submission_accuracy_python'] = float(sa.group(5))
        problemInfo['submission_accuracy_overall'] = float(sa.group(6))
        
        # grab problems failed by challenge
        pfbc = re.search('problems failed by challenge\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['problems_failed_by_challenge_java'] = int(pfbc.group(1))
        problemInfo['problems_failed_by_challenge_c++'] = int(pfbc.group(2))
        problemInfo['problems_failed_by_challenge_c#'] = int(pfbc.group(3))
        problemInfo['problems_failed_by_challenge_vb'] = int(pfbc.group(4))
        problemInfo['problems_failed_by_challenge_python'] = int(pfbc.group(5))
        problemInfo['problems_failed_by_challenge_overall'] = int(pfbc.group(6))
        
        # grab problems failed by system test
        pfbst = re.search('problems failed by system test\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['problems_failed_by_system_test_java'] = int(pfbst.group(1))
        problemInfo['problems_failed_by_system_test_c++'] = int(pfbst.group(2))
        problemInfo['problems_failed_by_system_test_c#'] = int(pfbst.group(3))
        problemInfo['problems_failed_by_system_test_vb'] = int(pfbst.group(4))
        problemInfo['problems_failed_by_system_test_python'] = int(pfbst.group(5))
        problemInfo['problems_failed_by_system_test_overall'] = int(pfbst.group(6))
        
        # grab challenge attempts made
        cam = re.search('challenge attempts made\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?\s+(\d*)?', t, i)
        problemInfo['challenge_attempts_made_java'] = int(cam.group(1))
        problemInfo['challenge_attempts_made_c++'] = int(cam.group(2))
        problemInfo['challenge_attempts_made_c#'] = int(cam.group(3))
        problemInfo['challenge_attempts_made_vb'] = int(cam.group(4))
        problemInfo['challenge_attempts_made_python'] = int(cam.group(5))
        problemInfo['challenge_attempts_made_overall'] = int(cam.group(6))
        
        # grab challenge accuracy
        ca = re.search('challenge accuracy\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?\s+(\d{1,3}\.\d{2})?%?', t, i)
        problemInfo['challenge_accuracy_java'] = float(ca.group(1))
        problemInfo['challenge_accuracy_c++'] = float(ca.group(2))
        problemInfo['challenge_accuracy_c#'] = float(ca.group(3))
        problemInfo['challenge_accuracy_vb'] = float(ca.group(4))
        problemInfo['challenge_accuracy_python'] = float(ca.group(5))
        problemInfo['challenge_accuracy_overall'] = float(ca.group(6))
        
        # grab best time
        try:
            bt = re.search('best time\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})', t, i)
            problemInfo['best_time_java'] = bt.group(1)
            problemInfo['best_time_c++'] = bt.group(2)
            problemInfo['best_time_c#'] = bt.group(3)
            problemInfo['best_time_vb'] = bt.group(4)
            problemInfo['best_time_python'] = bt.group(5)
            problemInfo['best_time_overall'] = bt.group(6)
        except: # no best time for any category
            problemInfo['best_time_java'] = None
            problemInfo['best_time_c++'] = None
            problemInfo['best_time_c#'] = None
            problemInfo['best_time_vb'] = None
            problemInfo['best_time_python'] = None
            problemInfo['best_time_overall'] = None
        
        # grab average correct time
        try:
            act = re.search('average correct time\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})?\s+(\d:\d{2}:\d{2}\.\d{1,3})', t, i)
            problemInfo['average_correct_time_java'] = act.group(1)
            problemInfo['average_correct_time_c++'] = act.group(2)
            problemInfo['average_correct_time_c#'] = act.group(3)
            problemInfo['average_correct_time_vb'] = act.group(4)
            problemInfo['average_correct_time_python'] = act.group(5)
            problemInfo['average_correct_time_overall'] = act.group(6)
        except: # whole line is empty
            problemInfo['average_correct_time_java'] = None
            problemInfo['average_correct_time_c++'] = None
            problemInfo['average_correct_time_c#'] = None
            problemInfo['average_correct_time_vb'] = None
            problemInfo['average_correct_time_python'] = None
            problemInfo['average_correct_time_overall'] = None
            
    coll.insert(problemInfo)
        

In [ ]:
import requests, json, re, pprint, pymongo
from bs4 import BeautifulSoup

client = pymongo.MongoClient (host="da0.eecs.utk.edu")
db = client['TopCoder']
coll = db['challenges']

count = 0
for rowObj in coll.find({}):
    print(type(rowObj))
    break
print(count)